In [1]:
from binary_tree import *

class BinarySearchTree(BinaryTree):
    def insert(self, data):
        #삽입할 노드 생성 및 데이터 설정
        new_node = self.make_node()
        self.set_node_data(new_node, data)

        cur = self.get_root()
        #루트 노드가 없을 때
        if cur == None:
            self.set_root(new_node)
            return

        #삽입할 노드의 위치를 찾아 삽입
        while True:
            #삽입할 데이터가 현재 노드 데이터보다 작을 때
            if data < self.get_node_data(cur):
                #왼쪽 자식 노드 존재하면
                if self.get_left_sub_tree(cur):
                    cur = self.get_left_sub_tree(cur)
                #존재하지 않으면 노드 삽입
                else:
                    self.make_left_sub_tree(
                        cur, new_node)
                    break
            #삽입할 데이터가 현재 노드 데이터보다 클 때
            elif data > self.get_node_data(cur):
                #오른쪽 자식 노드 존재하면
                if self.get_right_sub_tree(cur):
                    cur = self.get_right_sub_tree(cur)
                #존재하지 않으면 노드 삽입
                else:
                    self.make_right_sub_tree(
                        cur, new_node)
                    break

    def search(self, target):
        cur = self.get_root()
        
        while cur != None:
            #target 데이터를 찾으면 노드를 반환
            if target == self.get_node_data(cur):
                return cur
            #target 데이터가 노드 데이터보다 작으면
            #왼쪽 자식 노드로 이동
            elif target < self.get_node_data(cur):
                cur = self.get_left_sub_tree(cur)
            #target 데이터가 노드 데이터보다 크면
            #오른쪽 자식 노드로 이동
            elif target > self.get_node_data(cur):
                cur = self.get_right_sub_tree(cur)
        return cur
       
    def remove(self, target):
        del_parent = self.get_root()
        del_node = self.get_root()
        # 처음 시작은 지우려는 노드도, 지우려는 노드의 부모 노드도 최상위 루트라고 가정하고 출발
        
        # del_node가 target 을 가진 노드를 가르키게
        # del_parent는 del_node의 부모를 가르키게
        # 만약 target을 못찾으면 None으로 리턴
        while True:
            if not del_node:
                return None
                # 지우려는게 없으니 지우는 작업도 없어서 함수 자체를 빠져나가도 된다
            
            if target == self.get_node_data(del_node):
                break  # 아직 지우는 작업이 남았기 때문에 while문만 빠져나가야지 함수를 빠져나감 안된당
            elif target < self.get_node_data(del_node):
                del_parent = del_node
                del_node = self.get_left_sub_tree(del_node)
            else:
                del_parent = del_node
                del_node = self.get_right_sub_tree(del_node)
                
        # 1. del_node가 leaf_node일 때        
        if not self.get_left_sub_tree(del_node) and not self.get_right_sub_tree(del_node):
            return self.remove_leaf(del_parent,del_node)
        # 2. del_node의 자식 노드가 하나일 때
        elif not self.get_left_sub_tree(del_node) or not self.get_right_sub_tree(del_node):
            return self.remove_one_child(del_parent,del_node)
        # 3. del_node의 자식 노드가 두개일 때
        else:
            return self.remove_two_children(del_node)

    # remove 함수에 들어갈 세 종류의 지우기 방식
    # 1. del_node가 리프노드일 때
    def remove_leaf(self, parent, del_node):
        #del_node가 리프 노드이자 루트 노드일 때
        if del_node == self.get_root():
            self.self_root(None)
            return del_node
            
        #del_node가 parent의 왼쪽 노드일 때
        if self.get_left_sub_tree(parent) == del_node:
            self.make_left_sub_tree(parent,None)
        
        #del_node가 parent의 오른쪽 노드일 때
        else:
            self.make_right_sub_tree(parent,None)
        return del_node

        
    # 2. del_node의 자식 노드가 하나일 때
    #del_child가 del_node의 하나인 자식을 가르키게 하고
    #만약 parent의 왼쪽이 del_node면 parent의 왼쪽에 del_node대신 del child를 연결
    #만약 parent의 오른쪽이 del_node면 parent의 오른쪽에 del_node대신 del child를 연결
    def remove_one_child(self, parent, del_node):
        # del_node가 root 노드일 때
        if self.get_root() ==del_node:
            if self.get_left_sub_tree(del_node):
                self.set_root(self.get_left_sub_tree(del_node))
            else:
                self.set_root(self.get_right_sub_tree(del_node))
        del_child=None
        
        # del_node가 root 노드가 아닐 때
        
        # del 노드와 자식노드 관계
        # 자식 노드가 왼쪽에 있을 때
        if self.get_left_sub_tree(del_node):
            del_child = self.get_left_sub_tree(del_node)
        else:
            del_child = self.get_right_sub_tree(del_node)
        
        # del 노드와 부모노드 관계
        if self.get_left_sub_tree(parent) == del_node:
            self.make_left_sub_tree(parent,del_child)
        else:
            self.make_right_sub_tree(parent,del_child)
        
        return del_node
    
    
    
    # 3. del_node의 자식 노드가 두 개일 때
    #### 중요!!! 이 경우엔 노드를 삭제하는게 아니라 대체노드와 값을 바꾼 후 대체노드를 대신 삭제한다
    # 대체노드는 왼쪽노드에서 가장 큰 노드를 찾기로 하자
    def remove_two_children(self, del_node):
        #예외처리 필요 없다
        
        replace_parent = del_node
        replace = self.get_left_sub_tree(del_node)
        while self.get_right_sub_tree(replace):
            replace_parent = replace
            replace = self.get_right_sub_tree(replace_parent)
        
        del_data = self.get_node_data(del_node)
        self.set_node_data(del_node,self.get_node_data(replace))
        self.set_node_data(replace,del_data)
        
        if self.get_left_sub_tree(replace_parent) ==replace:
            self.make_left_sub_tree(del_node,self.get_left_sub_tree(replace))
        else:
            self.make_right_sub_tree(replace_parent,self.get_left_sub_tree(replace))
        
        return replace

    
    
    
    


In [2]:
if __name__ == "__main__":
    bst = BinarySearchTree()
    
    #insert
    bst.insert(6)
    bst.insert(3)
    bst.insert(2)
    bst.insert(4)
    bst.insert(5)
    bst.insert(8)
    bst.insert(10)
    bst.insert(9)
    bst.insert(11)

    f = lambda x: print(x, end = '  ')
    #전위 순회
    bst.preorder_traverse(bst.get_root(), f)
    print("")
    
    #search
    print("searched data : {}".format(bst.search(8).data))

    #remove - 1 : 리프 노드일 때
    #bst.remove(9)

    #remove - 2 : 자식 노드 하나일 때
    #bst.remove(8)

    #remove - 3: 자식 노드 두 개일 때
    bst.remove(6)
    
    bst.preorder_traverse(bst.get_root(), f)


6  3  2  4  5  8  10  9  11  
searched data : 8
TreeNode of 6 is deleted
5  3  2  4  8  10  9  11  